In [11]:
import os  # 导入os模块

from langchain.llms import ChatGLM  # 导入ChatGLM类
from langchain.document_loaders import DirectoryLoader  # 导入DirectoryLoader类
from langchain.text_splitter import CharacterTextSplitter  # 导入CharacterTextSplitter类
import gradio as gr  # 导入gradio库
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings  # 导入HuggingFaceBgeEmbeddings类
from langchain.vectorstores import Chroma  # 导入Chroma类

In [10]:
# 去命令行运行 python api.py

In [12]:
# 设置端点URL和ChatGLM对象的参数
endpoint_url = "http://127.0.0.1:8000"  # 设定端点URL
llm = ChatGLM(endpoint_url=endpoint_url,  # 创建ChatGLM对象
              max_token=80000,  # 设定最大令牌数
              top_p=0.9)  # 设定top_p值

In [13]:
def load_documents(directory="books"):
    # 创建DirectoryLoader对象并加载文档
    loader = DirectoryLoader(directory)  
    print(11)  # 打印信息
    documents = loader.load()  # 加载文档
    # 创建CharacterTextSplitter对象并拆分文档
    text_spliter = CharacterTextSplitter(chunk_size=512, chunk_overlap=5)  
    split_docs = text_spliter.split_documents(documents)  # 拆分文档
    return split_docs  # 返回拆分后的文档


def load_embedding_mode():
    encode_kwargs = {"normalize_embeddings": False}  # 编码参数
    model_kwargs = {"device": "cuda:0"}  # 模型参数
    # 创建HuggingFaceBgeEmbeddings对象
    return HuggingFaceBgeEmbeddings(model_name="ernie", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)


def store_chroma(docs, embeddings, persist_directory="VectorStore"):
    # 创建Chroma对象并持久化
    db = Chroma.from_documents(docs, embeddings, persist_directory=persist_directory)  
    db.persist()  # 持久化
    return db  # 返回Chroma对象

In [14]:
# 如果"VectorStore"目录不存在，则加载文档并存储Chroma数据库；否则直接加载Chroma数据库
embeddings = load_embedding_mode()  # 加载嵌入模型
if not os.path.exists("VectorStore"):  # 如果"VectorStore"目录不存在
    documents = load_documents()  # 加载文档
    db = store_chroma(documents, embeddings)  # 存储Chroma数据库
else:
    db = Chroma(persist_directory="VectorStore", embedding_function=embeddings)  # 直接加载Chroma数据库


No sentence-transformers model found with name ernie. Creating a new one with MEAN pooling.


In [15]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
# 创建qa
QA_CHAIN_PROMPT = PromptTemplate.from_template("""根据下面的上下文（context）内容回答问题。
如果你不知道答案，就回答不知道，不要试图编造答案。
答案最多3句话，保持答案简介。
总是在答案结束时说”谢谢你的提问！“
{context}
问题：{question}
""")
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [16]:
#测试一下
response = qa.invoke("小王子为什么离开玫瑰")
print(response)



> Entering new RetrievalQA chain...

> Finished chain.
{'query': '小王子为什么离开玫瑰', 'result': '小王子离开玫瑰是因为他意识到玫瑰对他的感情是虚伪的，他想要寻找真正的友谊和爱情。在离开的过程中，小王子也学会了珍惜自己的时间和真实的人际关系。'}


In [17]:
def chat(question, history):
    response = qa.invoke(question)  # 调用qa对象进行问答
    return response  # 返回回答

demo = gr.ChatInterface(chat)  # 创建ChatInterface对象
demo.launch(inbrowser=True, share=True)  # 启动交互式聊天界面，可以在浏览器中访问并共享


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: /usr/local/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.10/site-packages/gradio


No sentence-transformers model found with name ernie. Creating a new one with MEAN pooling.


In [40]:
response = qa.invoke("小王子为什么离开玫瑰")
print(response)



> Entering new RetrievalQA chain...

> Finished chain.
{'query': '小王子为什么离开玫瑰', 'result': '小王子离开玫瑰是因为他意识到玫瑰对他的感情是依赖的，而不是真正爱他。他不想成为玫瑰的负担，希望玫瑰能够快乐地生活在他所爱的星球上。同时，小王子也渴望探索更广阔的宇宙，寻找真正的友谊和爱情。'}


Running on local URL:  http://127.0.0.1:7861

Could not create share link. Missing file: /usr/local/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.10/site-packages/gradio


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  d



Download which package (l=list; x=cancel)?


  Identifier>  l


Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] bcp47............... BCP-47 Language Tags
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [ ] cess_esp............ CESS-ESP Treebank
  [ ] chat80.............. Chat-80 Data Files
  [ ] city_database....... City Database
  [ ] cmudict............. The Carnegie Mellon Pronouncing Dictionary (0.6)
  [ ] comparative_sentences Comparative Sentence Dataset
  [

...


Download which package (l=list; x=cancel)?


  Identifier>  all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger is already up-to-date!
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger_ru is already up-to-
       |       date!
       | Downloading package basque_grammars to /root/nltk_data...
       |   Package basque_grammars is already up-to-date!
       | Downloading package bcp47 to /root/nltk_data...
       |   Package bcp47 is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Packag


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
